<a href="https://colab.research.google.com/github/tomalexsmith/Bitcoin-sentiment-analysis/blob/main/sentiment%20analysis%20implementation/Naive_Bayes.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.preprocessing import FunctionTransformer
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import MultinomialNB, GaussianNB
from sklearn.pipeline import Pipeline
from nltk.stem import WordNetLemmatizer
from nltk.tokenize import TweetTokenizer
from nltk.corpus import stopwords
from nltk.tag import pos_tag
import pandas as pd

In [ ]:
from google.colab import drive

drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
path = '/content/drive/My Drive/preprocessedCorpus.csv'

with open(path, 'r', encoding = 'latin-1') as f:
  tweets_df = pd.read_csv(f, index_col=0)

tweets_df.head(len(tweets_df))

,polarity_label,content
0,Positive,devilsnight hey bug martini glass
1,Positive,lumifish prejudice
2,Negative,bed get drive dallas tomorrow
3,Negative,get feeling job hunt never come close
4,Negative,man damn weather suppose oahu right
...,...,...
499995,Positive,mikerelm show granada last night sick del hard...
499996,Negative,cannot find telephone
499997,Positive,littledotty enjoy cuppa go put kettle
499998,Negative,finally sidekick stuff yay go sf pride weekend...


In [ ]:
content=tweets_df['content']
polarity=tweets_df['polarity_label']

In [ ]:
from sklearn.model_selection import train_test_split

# Split datasets into training, testing, and validation datasets

X_train, X_val_test, y_train, y_val_test = train_test_split(content,
                                                            polarity,
                                                            random_state=42,
                                                            train_size=.8
                                                            )

X_val, X_test, y_val, y_test = train_test_split(X_val_test,
                                                y_val_test,
                                                random_state=42,
                                                train_size=.5
                                                )

In [ ]:
# Implement TF-ID vectorisation
tfid = TfidfVectorizer(max_features=100000, ngram_range=(2, 2))

X_train_transformed = tfid.fit_transform(X_train.values.astype('U'))
X_test_transformed = tfid.fit_transform(X_test.values.astype('U'))

In [ ]:
from sklearn.preprocessing import LabelEncoder

# Convert sentiment polarity labels to binary format
encoder = LabelEncoder()
encoder.fit(tweets_df['polarity_label'].to_list())

y_train = encoder.transform(y_train.to_list())

In [ ]:
nb_model=MultinomialNB(alpha=1)
nb_model.fit(X_train_transformed, y_train)

MultinomialNB(alpha=1)

In [ ]:
data = {'Actual polarity': encoder.transform(y_test.to_list()),
        'Content': X_test}


results = pd.DataFrame(data)
results = results.dropna()

In [ ]:
# Predict sentiment polarity of dataset
results['Predicted_polarity'] = nb_model.predict(tfid.fit_transform(results['Content'].values.astype('U')))

In [ ]:
from sklearn.metrics import accuracy_score, recall_score, precision_score, f1_score

#Calculate performance metrics
def calculate_metrics(test_var, pred_var):

  accuracy = accuracy_score(test_var, pred_var)
  recall = recall_score(test_var, pred_var, average='macro')
  precision = precision_score(test_var, pred_var, average='macro')
  f1 = f1_score(test_var, pred_var, average='macro')

  return accuracy, precision, recall, f1

accuracy, precision, recall, f1 = calculate_metrics(results["Actual polarity"], results["Predicted_polarity"])

print("Accuracy:", accuracy)
print("Recall:", recall)
print("Precision:", precision)
print("F1:", f1)

Accuracy: 0.5034615230701509
Recall: 0.5035450132580601
Precision: 0.5035670990343772
F1: 0.5027322058730397


In [ ]:
results.head(len(results))

,Actual polarity,Content,Predicted_polarity
32134,0,clairenick lose follower laugh loud miss clair...,1
469167,0,think I loose,0
59371,0,another dead turtle one last week half,1
427358,1,dwighthoward congratulation guy awesome,1
183371,1,laurenconrad cute lovely,0
...,...,...,...
378588,1,caitra really ah man end summer approaching,0
67919,0,sabrinabryan booo fail,0
428822,1,akr nokia green room,0
356555,0,friday almost follower today guy like,1


In [ ]:
# Create new dataframe containing performance metrics results
comparison_metrics = pd.DataFrame({'Accuracy':[accuracy],
                                  'Recall':[recall],
                                  'Accuracy':[precision],
                                  'F1':[f1]},
                                   index=['Naive Bayes']
                                  )

path = '/content/drive/My Drive/comparison_metrics.csv'

with open(path, 'a', encoding = 'utf-8') as f:
  comparison_metrics.to_csv(f, header=False)